In [3]:
from collections import Counter
from topicnet.cooking_machine.dataset import dataset2counter



In [4]:
from topicnet.cooking_machine import Dataset


In [5]:
from topicnet.embeddings.keyed_vectors import (
    get_doc_vec_phi, get_doc_vec_keyedvectors, topic_model_to_keyed_vectors, calc_dataset_statistics
)
from topicnet.cooking_machine.models import TopicModel


/data/mytopicnet/lib/python3.8/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


## load model

In [2]:
from topicnet.embeddings.api import load_model

any_model = load_model("ARTM_150_Base")

100%|██████████| 42.0M/42.0M [00:00<00:00, 96.9MiB/s]


## word embeddings

Векторные представления слов позволяют выразить семантическую похожесть/связь слов через пространственные отношения между векторами. 

In [6]:
keyed_vectors = topic_model_to_keyed_vectors(any_model, "@lemmatized")

Пример векторного представления слова "библия" (то есть эмбеддинг слова "библия" )

In [7]:
keyed_vectors.get_vector("библия")

array([9.9459379e-05, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 5.7174258e-15, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       1.6374557e-08, 4.9805258e-05, 9.1217656e-16, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 8.0916889e-06,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.2708416e-04,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.5917989e-05,
       1.3626588e-11, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       1.1377406e-05, 0.0000000e+00, 1.7141596e-04, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       2.3118336e-14, 4.4071062e-09, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 2.8508982e-15, 0.0000000e+00,
       0.0000000e+00, 3.0577880e-07, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 8.2948043e

В контексте тематических эмбеддингов, построенных на основе статей Википедии, два слова будут близки в случае, если они часто встречаются в похожих статьях.

In [56]:
# оба этих слова связаны со статьями Википедии про различные персоналии

keyed_vectors.similarity("родиться", "фильмография")

0.33605382

In [49]:
keyed_vectors.similarity("космос", "галактика")

0.026072497

In [48]:
keyed_vectors.similarity("валюта", "биржа")

0.005496593

In [59]:
keyed_vectors.similarity("сцена", "польша")

8.7197375e-05

Пример задачи, решаемой эмбеддингами слов: определение лишнего слова в наборе

In [61]:
keyed_vectors.doesnt_match(["жена", "год", "встреча", "орден", "интерференция"])

'интерференция'

In [62]:
keyed_vectors.doesnt_match(["паук", "муха", "млекопитающее", "церковь"])

'церковь'

In [63]:
keyed_vectors.doesnt_match(["паук", "муха", "млекопитающее"])

'млекопитающее'

In [64]:
keyed_vectors.doesnt_match(["сингл", "концерт", "чарт", "микроскоп"])

'микроскоп'

## document embeddings

### document embedding via averaging

Векторные представления документов также полезны для прикладных задач. Рассмотрим задачу обнаружения парафраз (http://paraphraser.ru): по двум новостным заголовкам требуется определить, говорят ли они об одном и том же событии.

In [65]:

dataset_paraphraser = Dataset("/home/bulatov/nelder_mead/paraphraser_dataset.csv",  internals_folder_path="/home/bulatov/nelder_mead/paraphraser_dataset")
paraphraser_counter = dataset2counter(dataset_paraphraser)
dict_parap = calc_dataset_statistics(dataset_paraphraser)

In [66]:
docs = [
    (
        Counter({'задержать': 1, 'в': 1, 'москва': 1, 'американский': 1, 'шпион': 1, 'скрываться': 1, 'изменять': 1, 'внешность': 1}), 
        Counter({'опубликовать': 1, 'видео': 1, 'задержание': 1, 'американский': 1, 'шпион': 1, 'в': 1, 'москва': 1}),
        0
    ),

    (
        Counter({'синоптик': 1, 'зафиксировать': 1, 'в': 1, 'москва': 1, 'рекордный': 1, 'сугроб': 1}),
        Counter({'на': 1, 'москва': 1, 'выпасть': 1, '0,1': 1, 'зима': 1}),
        -1
    ),

    (
        Counter({'законопроект': 1, 'о': 1, 'национализация': 1, 'имущество': 1, 'рф': 1, 'на': 1, 'украина': 1, 'внести': 1, 'в': 1, 'раду': 1}),
        Counter({'в': 1, 'раду': 1, 'внести': 1, 'законопроект': 1, 'о': 1, 'национализация': 1, 'имущество': 1, 'россия': 1}),
        1
    )
]

Векторные представления документов можно получить усреднением эмбеддингов всех слов, содержающихся в нём. Рассмотрим две схемы усреднения: с одинаковыми весами или усреднение на основе статистики вхождения слова в разные документы.

In [67]:
from sklearn.metrics.pairwise import cosine_similarity
dict_df = dict_parap
vec = keyed_vectors

for pair in docs:
    res = {}
    for avg_scheme in ['unit', "tf-idf"]:
        v1 = get_doc_vec_keyedvectors(vec, pair[0], dict_df, avg_scheme).values.reshape(1, -1)
        v2 = get_doc_vec_keyedvectors(vec, pair[1], dict_df, avg_scheme).values.reshape(1, -1)
        predicted_val = cosine_similarity(v1, v2)[0][0]
        res[avg_scheme] = predicted_val
    print()
    print("true_label = ", pair[2])
    print(pair[0])
    print(pair[1])
    print(res)




true_label =  0
Counter({'задержать': 1, 'в': 1, 'москва': 1, 'американский': 1, 'шпион': 1, 'скрываться': 1, 'изменять': 1, 'внешность': 1})
Counter({'опубликовать': 1, 'видео': 1, 'задержание': 1, 'американский': 1, 'шпион': 1, 'в': 1, 'москва': 1})
{'unit': 0.31887765811145047, 'tf-idf': 0.23546726628676873}

true_label =  -1
Counter({'синоптик': 1, 'зафиксировать': 1, 'в': 1, 'москва': 1, 'рекордный': 1, 'сугроб': 1})
Counter({'на': 1, 'москва': 1, 'выпасть': 1, '0,1': 1, 'зима': 1})
{'unit': 0.3715961955866923, 'tf-idf': 0.17331172489455154}

true_label =  1
Counter({'законопроект': 1, 'о': 1, 'национализация': 1, 'имущество': 1, 'рф': 1, 'на': 1, 'украина': 1, 'внести': 1, 'в': 1, 'раду': 1})
Counter({'в': 1, 'раду': 1, 'внести': 1, 'законопроект': 1, 'о': 1, 'национализация': 1, 'имущество': 1, 'россия': 1})
{'unit': 0.37454990501442825, 'tf-idf': 0.6490250998283357}


Усреднение с весами tf-idf делает эмбеддинг документа более устойчивым к частым, но неинформативным словам и помогает подстроить эмбеддинги документов под коллекцию с нетипичной лекской. 

Сходство первой и второй документовой пары уменьшилось, что соответствует меткам "0" и "-1". Третья пара документов, напротив, увеличила своё сходство (что соответствует метке "+1").

Проверим, что разные стратегии усреднения действительно влияют на полученные документные эмбеддинги.

In [68]:
# different averaging has an influence on the vectors and their cosine distance


for avg_scheme in ['unit', "tf-idf"]:
    v1 = get_doc_vec_keyedvectors(vec, pair[0], dict_df, avg_scheme).values.reshape(1, -1)
    v2 = get_doc_vec_keyedvectors(vec, pair[1], dict_df, avg_scheme).values.reshape(1, -1)
    predicted_val = cosine_similarity(v1, v2)[0][0]
    print(avg_scheme, predicted_val)
    print(sum(v1[0]), sum(v2[0]))
    print(max(v1[0]), max(v2[0]))




unit 0.37454990501442825
0.013826804489552768 0.007170047072100804
0.005285812316287775 0.0017499953202786856
tf-idf 0.6490250998283357
0.08219843898783698 0.06043779218852825
0.021489167828500893 0.015217080430011265
